In [2]:
import sys
import subprocess
import os
import matplotlib
import matplotlib.pyplot as plt
import pylab
import pandas as pd
from xml.etree import ElementTree as ET

pylab.rcParams['figure.figsize'] = (12, 6)
pd.options.display.max_rows = 10

os.chdir("d:/gygCode/myORE")
ore_exe = "d:/gygCode/myORE/ore.exe"
print('python enviroment:', os.path.abspath('.'))

python enviroment: d:\gygCode\myORE


In [3]:
from portfolio import Portfolios

Portfolios(True).add('IRSNS', tradeID='Leg001', Notional =100, Coupon=0.009, startD='20160209', endD='20190209')
Portfolios(False).add('FXAmerican',tradeID='Leg002', Notional= 2000)
Portfolios(False).add('FXAsian',tradeID='Leg003', Notional= 2000)
Portfolios(False).add('FXEuropean',tradeID='Leg004', Notional= 2000)
Portfolios(False).add('MultipleRanges',tradeID='Leg005', Notional= 2000)


In [4]:
initialdata = "Input/0.ore.xml"

root = ET.parse(initialdata).getroot()

root.find('Setup/Parameter[@name ="portfolioFile"]').text = '1.portfolio.xml'
root.find('Analytics/Analytic[@type ="npv"]').find('Parameter[@name="active"]').text = 'Y'
root.find('Analytics/Analytic[@type ="npv"]').find('Parameter[@name="outputFileName"]').text = 'out_npv.csv'
root.find('Analytics/Analytic[@type ="cashflow"]').find('Parameter[@name="active"]').text = 'Y'
root.find('Analytics/Analytic[@type ="sensitivity"]').find('Parameter[@name="active"]').text = 'Y'
root.find('Analytics/Analytic[@type ="simulation"]').find('Parameter[@name="active"]').text = 'N'
root.find('Analytics/Analytic[@type ="xva"]').find('Parameter[@name="active"]').text = 'N'
root.find('Analytics/Analytic[@type ="parametricVar"]').find('Parameter[@name="active"]').text = 'N'

ET.ElementTree(root).write(initialdata, encoding='utf-8', xml_declaration=True)

res = subprocess.call([ore_exe, initialdata])
print('res:', res)

res: 0


In [5]:
npvdata = pd.read_csv('Output/out_npv.csv')
npvdata = npvdata[['#TradeId','TradeType','Maturity','NPV','NpvCurrency','NPV(Base)','BaseCurrency']]
display('npv:',npvdata)


'npv:'

,#TradeId,TradeType,Maturity,NPV,NpvCurrency,NPV(Base),BaseCurrency
0,Leg001,Swap,2019-02-11,-0.367053,USD,-0.367053,USD
1,Leg002,ScriptedTrade,2017-02-08,97.045218,USD,97.045218,USD
2,Leg003,ScriptedTrade,2017-02-06,56.330547,USD,56.330547,USD
3,Leg004,ScriptedTrade,2017-02-07,94.566767,USD,94.566767,USD
4,Leg005,ScriptedTrade,2017-02-07,55.226420,USD,55.226420,USD


In [6]:
cashflowdata = pd.read_csv('Output/flows.csv')
cashflowdata = cashflowdata[['#TradeId', 'Type', 'PayDate','FlowType', 'Amount', 'Currency']]
display('cashflow:',cashflowdata)

'cashflow:'

,#TradeId,Type,PayDate,FlowType,Amount,Currency
0,Leg001,Swap,2016-05-09,Interest,0.2250,USD
1,Leg001,Swap,2016-08-09,Interest,0.2250,USD
2,Leg001,Swap,2016-11-09,Interest,0.2250,USD
3,Leg001,Swap,2017-02-09,Interest,0.2250,USD
4,Leg001,Swap,2017-05-09,Interest,0.2250,USD
...,...,...,...,...,...,...
20,Leg001,Swap,2018-05-09,InterestProjected,-0.2974,USD
21,Leg001,Swap,2018-08-09,InterestProjected,-0.3074,USD
22,Leg001,Swap,2018-11-09,InterestProjected,-0.3074,USD
23,Leg001,Swap,2019-02-11,InterestProjected,-0.3141,USD


In [7]:
greeksdata = pd.read_csv('Output/sensitivity.csv')
display('greeks:',greeksdata)
#display('Expected Exposure:')
#data = pd.read_csv('example/example2/Output/exposure_trade_Swap_20.csv')
#plt.plot(data['Time'], data[['EPE','ENE']]), plt.legend(['EPE','ENE']),plt.title('Expected Exposure'),plt.show();

'greeks:'

,#TradeId,IsPar,Factor_1,ShiftSize_1,Factor_2,ShiftSize_2,Currency,Base NPV,Delta,Gamma
0,Leg001,False,DiscountCurve/USD/0/6M,0.0001,NaN,0.0000,USD,-0.37,0.0,0.0
1,Leg001,False,DiscountCurve/USD/1/1Y,0.0001,NaN,0.0000,USD,-0.37,0.0,0.0
2,Leg001,False,DiscountCurve/USD/2/2Y,0.0001,NaN,0.0000,USD,-0.37,0.0,0.0
3,Leg001,False,DiscountCurve/USD/3/3Y,0.0001,NaN,0.0000,USD,-0.37,0.0,0.0
4,Leg001,False,IndexCurve/USD-LIBOR-3M/0/6M,0.0001,NaN,0.0000,USD,-0.37,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
9,Leg001,False,DiscountCurve/USD/2/2Y,0.0001,IndexCurve/USD-LIBOR-3M/1/1Y,0.0001,USD,-0.37,0.0,0.0
10,Leg001,False,DiscountCurve/USD/2/2Y,0.0001,IndexCurve/USD-LIBOR-3M/3/3Y,0.0001,USD,-0.37,0.0,0.0
11,Leg001,False,DiscountCurve/USD/3/3Y,0.0001,IndexCurve/USD-LIBOR-3M/2/2Y,0.0001,USD,-0.37,0.0,0.0
12,Leg001,False,DiscountCurve/USD/3/3Y,0.0001,IndexCurve/USD-LIBOR-3M/3/3Y,0.0001,USD,-0.37,0.0,0.0


In [8]:
initialdata = "Input/0.ore.xml"

root = ET.parse(initialdata).getroot()

root.find('Analytics/Analytic[@type ="sensitivity"]').find('Parameter[@name="active"]').text = 'Y'
root.find('Analytics/Analytic[@type ="parametricVar"]').find('Parameter[@name="active"]').text = 'Y'
root.find('Analytics/Analytic[@type ="parametricVar"]').find('Parameter[@name="sensitivityInputFile"]').text = '../Output/sensitivity.csv'
root.find('Analytics/Analytic[@type ="parametricVar"]').find('Parameter[@name="covarianceInputFile"]').text = '../Meta/covariance.csv'
ET.ElementTree(root).write(initialdata, encoding='utf-8', xml_declaration=True)

res = subprocess.call([ore_exe, initialdata])
print('res:', res)

vardata = pd.read_csv('Output/var.csv')
display('var:', vardata)


res: 0


'var:'

,#Portfolio,RiskClass,RiskType,Quantile_0.010000,Quantile_0.050000,Quantile_0.950000,Quantile_0.990000
0,All,All,All,-3.489522,-2.46728,2.46728,3.489522
1,PF1,All,All,-3.489522,-2.46728,2.46728,3.489522
2,All,All,DeltaGamma,-3.489522,-2.46728,2.46728,3.489522
3,PF1,All,DeltaGamma,-3.489522,-2.46728,2.46728,3.489522
4,All,InterestRate,All,-3.489522,-2.46728,2.46728,3.489522
5,PF1,InterestRate,All,-3.489522,-2.46728,2.46728,3.489522
6,All,InterestRate,DeltaGamma,-3.489522,-2.46728,2.46728,3.489522
7,PF1,InterestRate,DeltaGamma,-3.489522,-2.46728,2.46728,3.489522
